In [2]:
import fastf1
fastf1.Cache.enable_cache('../data/cache') 

In [3]:
# Set some variables
DRIVER = 'VER'
YEAR = 2022
RACE = 'Austria'

In [4]:
session = fastf1.get_session(YEAR, RACE, 'Race')
session.load(telemetry=True, laps=True, weather=False)

core           INFO 	Loading data for Austrian Grand Prix - Race [v2.2.9]
api            INFO 	Using cached data for driver_info
api            INFO 	Using cached data for timing_data
api            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
api            INFO 	Using cached data for session_status_data
api            INFO 	Using cached data for track_status_data
api            INFO 	Using cached data for car_data
api            INFO 	Using cached data for position_data
api            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['16', '1', '44', '63', '31', '47', '4', '20', '3', '14', '77', '23', '18', '24', '10', '22', '5', '55', '6', '11']


In [5]:
from IPython.display import display, Markdown, HTML
import pandas

In [6]:
# Fastest Lap Time

laps = session.laps.pick_driver(DRIVER)
fastest_lap = laps.pick_fastest()

display(Markdown('### Fastest Lap Time'))
print(fastest_lap["LapTime"].to_pytimedelta())

### Fastest Lap Time

0:01:07.275000


In [32]:
# Average Lap Time
# Excludes when pit stop occurred

average = laps.query('PitOutTime == "" and PitInTime == ""').filter(items=['LapTime']).mean()['LapTime']

display(Markdown('### Average Lap Time'))
print(average.round('ms').to_pytimedelta())

### Average Lap Time

0:01:09.983000


In [150]:
# Pit Stop Times
last_pit_in = None
pit_stops = []

_laps = laps.filter(items=['LapNumber', 'PitInTime', 'PitOutTime'])
for idx, lap in _laps.iterrows():
    lap_number = lap['LapNumber']
    pit_in = lap['PitInTime']
    pit_out = lap['PitOutTime']
    
    if pandas.isnull(pit_in) and not pandas.isnull(pit_out) and lap_number == 1:
        continue
    
    if not pandas.isnull(pit_in):
        last_pit_in = pit_in
    
    if not pandas.isnull(pit_out):
        pit_stops.append((lap_number-1, pandas.Timedelta(pit_out - last_pit_in)))
        last_pit_in = None
        
df = pandas.DataFrame(data=pit_stops, columns=['lap', 'time'])
average = df.filter(items=['time']).mean()['time'].round('ms').to_pytimedelta()
total = df.filter(items=['time']).sum()['time'].round('ms').to_pytimedelta()

display(Markdown('### Pit Stop Times'))
display(Markdown('#### Average'))
print(average)
display(Markdown('#### Total'))
print(total)
# For some reason these are slightly off from ergast pit information API.
# We consider pit time here as difference between when car entered the pit and exited.
# Maybe the API treats "pit duration" sligtly differently?
display(Markdown('#### By Lap'))
for lap, time in pit_stops:
    print(f'Lap {lap} - {time.round("ms").to_pytimedelta()}')


### Pit Stop Times

#### Average

0:00:21.598000


#### Total

0:01:04.793000


#### By Lap

Lap 13 - 0:00:22.008000
Lap 36 - 0:00:21.448000
Lap 58 - 0:00:21.337000


In [29]:
# Average Speed
display(Markdown('### Average Speed'))
display(Markdown('#### Fastest Lap'))

car_data = fastest_lap.get_car_data(pad=1, pad_side='both')
print(car_data.mean(numeric_only=True)['Speed'].round(3))

display(Markdown('#### All Laps'))
# Excludes pit stop
car_data = laps.query('PitOutTime == "" and PitInTime == ""').get_car_data(pad=1, pad_side='both')
print(car_data.mean(numeric_only=True)['Speed'].round(3))

### Average Speed

#### Fastest Lap

229.084


#### All Laps

215.953


In [10]:
# Speed Traps
display(Markdown('### Speed Traps'))

display(Markdown('#### Fastest Lap'))
display(Markdown('##### Intermediate 1'))
print(fastest_lap["SpeedI1"])
display(Markdown('##### Intermediate 2'))
print(fastest_lap["SpeedI2"])
display(Markdown('##### Speed Trap'))
print(fastest_lap["SpeedST"])
display(Markdown('##### Finising Line'))
print(fastest_lap["SpeedFL"])

display(Markdown('#### All Laps'))
df = laps.filter(items=['SpeedI1', 'SpeedI2', 'SpeedST', 'SpeedFL']).max()
display(Markdown('##### Intermediate 1'))
print(df["SpeedI1"])
display(Markdown('##### Intermediate 2'))
print(df["SpeedI2"])
display(Markdown('##### Speed Trap'))
print(df["SpeedST"])
display(Markdown('##### Finising Line'))
print(df["SpeedFL"])

### Speed Traps

#### Fastest Lap

##### Intermediate 1

305.0


##### Intermediate 2

...

238.0


##### Speed Trap

321.0


##### Finising Line

281.0


In [11]:
display(Markdown('### Sector Times'))

# Sector Average
df_avg = laps.filter(items=['Sector1Time', 'Sector2Time', 'Sector3Time']).mean().round('ms')
display(Markdown('#### Average'))
display(df_avg)

# Sector Fastest
df_max = laps.filter(items=['Sector1Time', 'Sector2Time', 'Sector3Time']).min().round('ms')
display(Markdown('#### Fastest'))
display(df_max)

### Sector Times

#### Average

Sector1Time   0 days 00:00:18.396000
Sector2Time   0 days 00:00:31.474000
Sector3Time   0 days 00:00:21.477000
dtype: timedelta64[ns]

#### Fastest

Sector1Time   0 days 00:00:17.086000
Sector2Time   0 days 00:00:29.866000
Sector3Time   0 days 00:00:20.265000
dtype: timedelta64[ns]

In [ ]:
this_driver = 1 # TODO
driver_ahead = None
threashold = pandas.Timedelta(3, 's')
pit_threashold = pandas.Timedelta(3, 's')

for i, lap in laps.iterlaps():
    lap_number = lap["LapNumber"]
    if lap_number == 1: # exclude first lap
        continue

    # Consider successful overtake as:
    # - driver ahead has changed at t0
    # - passed driver's driver ahead is now the driver's driver ahead any time between t0 + 3 seconds
    # - the driver is ahead by 1 car length
    # - the driver's driver ahead was the passed driver any time between t0 - 3 seconds
    # - driver has driven longer distance since t0 for the entirety of the sample window t0 + 3 seconds
    # - passed driver has not been lapped
    # - passed driver is not pitted
    # - passed driver is not off track
    # Consider adding checks for:
    # - track status (no yellow flags, sc leading to lapped cars making their way around).
    
    # Currently, if a car is side by side and the driver is slightly in front, gets away for a split second counts as an overtake.
    lap_telemetry = lap.get_car_data().add_driver_ahead()
    for _, t in lap_telemetry.iterrows():
        if driver_ahead == t['DriverAhead']:
            continue
 
        maybe_passed_driver = driver_ahead
        driver_ahead = t['DriverAhead']
        
        if not maybe_passed_driver:
            continue

        t0 = t['SessionTime']

        # Passed driver data
        pd_laps = session.laps.pick_driver(maybe_passed_driver)
        pd_lap_idx = pd_laps[(pd_laps['LapStartTime'] <= t0)]['LapStartTime'].idxmax()
        pd_lap = session.laps.iloc[pd_lap_idx]
        pd_pos = pd_laps.get_pos_data().slice_by_time(t0 - threashold, t0 + threashold)
        pd_tel = pd_laps.get_car_data()
        try:
            pd_tel_before = pd_tel[(pd_tel['SessionTime'] >= (t0 - threashold)) & (pd_tel['SessionTime'] < t0)].add_driver_ahead()
            pd_tel_after = pd_tel[(pd_tel['SessionTime'] >= t0) & (pd_tel['SessionTime'] < (t0 + threashold))].add_driver_ahead().add_distance()
        except (ValueError, IndexError):
            continue

        # Driver data
        driver_tel = session.laps.pick_driver(this_driver).get_car_data()
        try:
            driver_tel_before = driver_tel[(driver_tel['SessionTime'] >= (t0 - threashold)) & (driver_tel['SessionTime'] < t0)].add_driver_ahead()
            driver_tel_after = driver_tel[(driver_tel['SessionTime'] >= t0) & (driver_tel['SessionTime'] < (t0 + threashold))].add_driver_ahead().add_distance()
        except ValueError:
            continue
            
        # Ensure the car is not off track
        if not pd_pos.query(f'Status == "OffTrack"').empty:
            print("off track!!")
            continue
            
        # Check for possible overtake based on driver ahead during the sample windows
        if not(not pd_tel_before.query(f'DriverAhead == "{driver_ahead}"').empty and not pd_tel_after.query(f'DriverAhead == "{this_driver}"').empty):
            continue
            
        # The drive is at least 1 car length ahead (6 meters) at any point in time during t0 + 3 seconds
        if pd_tel_after.query(f'DriverAhead == "{this_driver}" and DistanceToDriverAhead >= 6').empty:
            continue
            
        # Check passed driver has not been lapped
        if pd_lap['LapNumber'] < lap_number:
            continue
    
        # Check the driver has driven longer distance since t0
        if not driver_tel_after.iloc[-1]['Distance'] > pd_tel_after.iloc[-1]['Distance']:
            continue
            
        # Check passed driver was not pitted
        # This is determined true if t0 does not occur during any PitInTime - 3 second and PitOutTime + 3 second
        # TODO: this could be optimized to filter laps we only care about
        pit_stops = []
        was_pitted = False
        for _, _pd_lap in pd_laps.iterrows():
            pit_in = _pd_lap['PitInTime']
            pit_out = _pd_lap['PitOutTime']

            if pandas.isnull(pit_in) and not pandas.isnull(pit_out) and lap_number == 1:
                continue

            if not pandas.isnull(pit_in):
                last_pit_in = pit_in

            if not pandas.isnull(pit_out):
                if t0 >= (last_pit_in - pit_threashold) and t0 <= (pit_out + pit_threashold):
                    was_pitted = True
                    break

        if was_pitted:
            continue

        print()
        print(f'--- Lap {lap_number} ---')
        print(f'-- Overtake --')
        display(pd_tel_after)
        display(t)
        print(f'Driver ahead before: {maybe_passed_driver}')
        print(f'Driver ahead now: {driver_ahead}')


--- Lap 16 ---
-- Overtake --


,Date,RPM,Speed,nGear,Throttle,Brake,DRS,Source,Time,SessionTime,DriverAhead,DistanceToDriverAhead,Distance
4115,2022-07-10 13:21:33.027,10354,291,8,100,False,0,car,0 days 00:18:19.220000,0 days 01:20:33.018000,,NaN,88853.616667
4116,2022-07-10 13:21:33.227,10344,290,8,99,False,0,car,0 days 00:18:19.420000,0 days 01:20:33.218000,1,1.575278,88869.727778
4117,2022-07-10 13:21:33.507,10294,289,8,100,False,0,car,0 days 00:18:19.700000,0 days 01:20:33.498000,1,3.441944,88892.205556
4118,2022-07-10 13:21:33.868,10289,289,8,99,False,0,car,0 days 00:18:20.061000,0 days 01:20:33.859000,1,5.848611,88921.185833
4119,2022-07-10 13:21:34.028,10191,288,8,100,False,0,car,0 days 00:18:20.221000,0 days 01:20:34.019000,1,7.004167,88933.985833
4120,2022-07-10 13:21:34.268,10178,286,8,100,False,0,car,0 days 00:18:20.461000,0 days 01:20:34.259000,1,8.937500,88953.052500
4121,2022-07-10 13:21:34.468,10161,284,8,100,False,0,car,0 days 00:18:20.661000,0 days 01:20:34.459000,1,10.659722,88968.830278
4122,2022-07-10 13:21:34.748,10150,284,8,100,False,0,car,0 days 00:18:20.941000,0 days 01:20:34.739000,1,12.604167,88990.919167
4123,2022-07-10 13:21:34.948,10083,284,8,85,True,0,car,0 days 00:18:21.141000,0 days 01:20:34.939000,1,12.881944,89006.696944
4124,2022-07-10 13:21:35.108,9803,266,8,0,True,0,car,0 days 00:18:21.301000,0 days 01:20:35.099000,1,12.881944,89018.519167


Date                     2022-07-10 13:21:33.027000
RPM                                           11037
Speed                                           310
nGear                                             8
Throttle                                        100
Brake                                         False
DRS                                              12
Source                                          car
Time                         0 days 00:00:16.096000
SessionTime                  0 days 01:20:33.018000
DriverAhead                                      44
DistanceToDriverAhead                    178.903611
Name: 57, dtype: object

Driver ahead before: 47
Driver ahead now: 44

--- Lap 18 ---
-- Overtake --


,Date,RPM,Speed,nGear,Throttle,Brake,DRS,Source,Time,SessionTime,DriverAhead,DistanceToDriverAhead,Distance
4704,2022-07-10 13:24:07.908,11070,282,7,100,False,0,car,0 days 00:20:54.101000,0 days 01:23:07.899000,,NaN,98237.911667
4705,2022-07-10 13:24:08.108,10531,284,8,100,False,0,car,0 days 00:20:54.301000,0 days 01:23:08.099000,1,1.011667,98253.689444
4706,2022-07-10 13:24:08.348,10594,287,8,100,False,0,car,0 days 00:20:54.541000,0 days 01:23:08.339000,1,1.811667,98272.822778
4707,2022-07-10 13:24:08.588,10638,287,8,100,False,0,car,0 days 00:20:54.781000,0 days 01:23:08.579000,1,2.811667,98291.956111
4708,2022-07-10 13:24:08.748,10698,290,8,100,False,0,car,0 days 00:20:54.941000,0 days 01:23:08.739000,1,3.389444,98304.845000
4709,2022-07-10 13:24:08.908,10714,291,8,100,False,0,car,0 days 00:20:55.101000,0 days 01:23:08.899000,1,4.011667,98317.778333
4710,2022-07-10 13:24:09.228,10743,292,8,100,False,0,car,0 days 00:20:55.421000,0 days 01:23:09.219000,1,5.433889,98343.733889
4711,2022-07-10 13:24:09.428,10743,293,8,100,False,0,car,0 days 00:20:55.621000,0 days 01:23:09.419000,1,6.378333,98360.011667
4712,2022-07-10 13:24:09.708,10785,294,8,100,False,0,car,0 days 00:20:55.901000,0 days 01:23:09.699000,1,7.778333,98382.878333
4713,2022-07-10 13:24:09.908,10778,294,8,100,False,0,car,0 days 00:20:56.101000,0 days 01:23:09.899000,1,8.945000,98399.211667


Date                     2022-07-10 13:24:07.908000
RPM                                           11533
Speed                                           294
nGear                                             7
Throttle                                        100
Brake                                         False
DRS                                              12
Source                                          car
Time                         0 days 00:00:30.592000
SessionTime                  0 days 01:23:07.899000
DriverAhead                                      55
DistanceToDriverAhead                    948.835833
Name: 114, dtype: object

Driver ahead before: 44
Driver ahead now: 55
